# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [31]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [33]:
#your code here
spaceship_cl=spaceship.dropna()
spaceship_cl.loc[:,'Cabin']= spaceship_cl['Cabin'].str[0].str.upper()
spaceship_cl=spaceship_cl.drop(columns = ["PassengerId","Name"])
spaceship_cl2 = pd.get_dummies(spaceship_cl, drop_first=False)
features2=spaceship_cl2.drop(columns = ["Transported"])
target2=spaceship_cl2["Transported"]

In [34]:
spaceship_cl2.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,False,True,False,...,False,False,False,False,False,False,False,True,True,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,True,False,False,...,False,False,True,False,False,False,False,True,True,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,False,True,False,...,False,False,False,False,False,False,False,True,False,True
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,False,True,False,...,False,False,False,False,False,False,False,True,True,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,True,False,False,...,False,False,True,False,False,False,False,True,True,False


**Perform Train Test Split**

In [35]:
#your code here
X_train2, X_test2, y_train2, y_test2 = train_test_split(features2, target2, test_size = 0.20, random_state=0)

In [36]:
normalizer = MinMaxScaler()
normalizer.fit(X_train2)
X_train_norm = normalizer.transform(X_train2)
X_test_norm = normalizer.transform(X_test2)
X_train_norm = pd.DataFrame(X_train_norm, columns=X_train2.columns, index=X_train2.index )
X_test_norm = pd.DataFrame(X_test_norm, columns=X_test2.columns, index=X_test2.index)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [37]:
#your code here Trying bagging with KNeighborsRegressor for comparison with previous lab

bagging_reg = BaggingRegressor(KNeighborsRegressor(),
                               n_estimators=100,
                               max_samples=1000)
bagging_reg.fit(X_train_norm, y_train2)


BaggingRegressor(estimator=KNeighborsRegressor(), max_samples=1000,
                 n_estimators=100)

In [38]:
y_pred_test_bag = bagging_reg.predict(X_train_norm)

print(f"R2 score {bagging_reg.score(X_test_norm, y_test2): .2f}")

R2 score  0.37


- Random Forests

In [41]:
#your code here
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)
forest.fit(X_train_norm, y_train2)

RandomForestRegressor(max_depth=20)

In [43]:
y_pred_test_forest = forest.predict(X_train_norm)

print(f"R2 score {forest.score(X_test_norm, y_test2): .2f}")

R2 score  0.41


- Gradient Boosting

In [44]:
#your code here
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)
gb_reg.fit(X_train_norm, y_train2)

GradientBoostingRegressor(max_depth=20)

In [45]:
y_pred_test_gb_reg = gb_reg.predict(X_train_norm)

print(f"R2 score {gb_reg.score(X_test_norm, y_test2): .2f}")

R2 score  0.29


- Adaptive Boosting

In [46]:
#your code here
ada_reg = AdaBoostRegressor(KNeighborsRegressor( p=1),
                            n_estimators=100)
ada_reg.fit(X_train_norm, y_train2)

AdaBoostRegressor(estimator=KNeighborsRegressor(p=1), n_estimators=100)

In [47]:
y_pred_test_ada_reg = ada_reg.predict(X_train_norm)

print(f"R2 score {ada_reg.score(X_test_norm, y_test2): .2f}")

R2 score  0.31


Which model is the best and why?